In [1]:
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 11.3 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.12-py3-none-any.whl size=102971 sha256=c03f6bf83855a1905004671d899f282da0df3d7267821ea4e14392633f5caa11
  Stored in directory: /Users/sanjaysampat/Library/Caches/pip/wheels/2a/00/63/fa8dbcfb1458e6a65ac7a28d456deb9e0e033245b67f952681
Successfully built kaggle


In [2]:
!kaggle datasets download -d yelp-dataset/yelp-dataset

Dataset URL: https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset
License(s): other
100%|█████████████████████████████████████▉| 4.07G/4.07G [01:11<00:00, 60.6MB/s]
100%|██████████████████████████████████████| 4.07G/4.07G [01:11<00:00, 60.8MB/s]


In [3]:
!unzip yelp-dataset.zip

Archive:  yelp-dataset.zip
  inflating: Dataset_User_Agreement.pdf  
  inflating: yelp_academic_dataset_business.json  
  inflating: yelp_academic_dataset_checkin.json  
  inflating: yelp_academic_dataset_review.json  
  inflating: yelp_academic_dataset_tip.json  
  inflating: yelp_academic_dataset_user.json  


In [9]:
import os
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Yelp Data Analysis") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/06 23:45:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### We first want to read in the datasets we are planning on using into a spark dataframe.

In [11]:
review_df = spark.read.json('yelp_academic_dataset_review.json')
business_df = spark.read.json('yelp_academic_dataset_business.json')
user_df = spark.read.json('yelp_academic_dataset_user.json') 


#### These datasets will eventually be joined on the attributes user_id for review and user and on business_id for review and business. However we can first explore the datasets individually to first understand what they have and maybe see what can get lost in the joins.

### Taking a closer look at the review dataframe to understand missingness and text statistics

In [13]:
review_df.printSchema()
review_df.show(8)
print("Number of Reviews: " + str(review_df.count()))

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPp

Number of Reviews: 6990280


In [ ]:
#looking for missing data wihtin the review section
from pyspark.sql.functions import col, count, when, length

missingdata_review = review_df.select([count(when(col(c).isNull(), c)).alias(c) for c in review_df.columns])
missingdata_review.show()


#looking for duplicates within the review section 
duplicate_review = review_df.groupby('text').count()\
    .where(col('count') >1).orderBy(col('count').desc())

duplicate_review.show()

+-----------+----+----+-----+---------+-----+----+------+-------+
|business_id|cool|date|funny|review_id|stars|text|useful|user_id|
+-----------+----+----+-----+---------+-----+----+------+-------+
|          0|   0|   0|    0|        0|    0|   0|     0|      0|
+-----------+----+----+-----+---------+-----+----+------+-------+



In [17]:
review_df = review_df.withColumn('review_length', length('text'))
review_df.describe('review_length').show()

+-------+-----------------+
|summary|    review_length|
+-------+-----------------+
|  count|          6990280|
|   mean|567.7644364746477|
| stddev|527.2578085249028|
|    min|                1|
|    max|             5000|
+-------+-----------------+



### Now we take a closer look at the business dataframe to once again understand missingness and all the features/attributes ther dataset offers

In [24]:
business_df.printSchema()
business_df.show(8)
print("Number of businesses: " + str(business_df.count()))

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [19]:
missingdata_business = business_df.select([count(when(col(c).isNull(), c)).alias(c) for c in business_df.columns]).show()
missingdata_business 

+-------+----------+-----------+----------+----+-----+-------+--------+---------+----+-----------+------------+-----+-----+
|address|attributes|business_id|categories|city|hours|is_open|latitude|longitude|name|postal_code|review_count|stars|state|
+-------+----------+-----------+----------+----+-----+-------+--------+---------+----+-----------+------------+-----+-----+
|      0|     13744|          0|       103|   0|23223|      0|       0|        0|   0|          0|           0|    0|    0|
+-------+----------+-----------+----------+----+-----+-------+--------+---------+----+-----------+------------+-----+-----+



In [20]:
business_df.groupBy('stars').count().orderBy('stars').show()

+-----+-----+
|stars|count|
+-----+-----+
|  1.0| 1986|
|  1.5| 4932|
|  2.0| 9527|
|  2.5|14316|
|  3.0|18453|
|  3.5|26519|
|  4.0|31125|
|  4.5|27181|
|  5.0|16307|
+-----+-----+



#### Exploring the different types of businesses and which are the most prevalent

In [21]:
from pyspark.sql.functions import explode, split
business_df.withColumn('category', explode(split('categories', ', '))).groupBy('category').count().orderBy('count', ascending=False).show()

+--------------------+-----+
|            category|count|
+--------------------+-----+
|         Restaurants|52268|
|                Food|27781|
|            Shopping|24395|
|       Home Services|14356|
|       Beauty & Spas|14292|
|           Nightlife|12281|
|    Health & Medical|11890|
|      Local Services|11198|
|                Bars|11065|
|          Automotive|10773|
|Event Planning & ...| 9895|
|          Sandwiches| 8366|
|American (Traditi...| 8139|
|         Active Life| 7687|
|               Pizza| 7093|
|        Coffee & Tea| 6703|
|           Fast Food| 6472|
|  Breakfast & Brunch| 6239|
|      American (New)| 6097|
|     Hotels & Travel| 5857|
+--------------------+-----+
only showing top 20 rows



#### We can group on the city to first see how many businesses on yelp are in each city and also for city-wide comparisons later on

In [22]:
business_df.groupBy('city').count().orderBy('city').show()

+--------------------+-----+
|                city|count|
+--------------------+-----+
|         AB Edmonton|    1|
|              AMBLER|    1|
|             ARDMORE|    1|
|                AVON|    1|
|            Abington|  169|
|   Abington Township|    3|
|              Affton|   36|
|               Afton|    1|
|Alberta Park Indu...|    1|
|               Aldan|   11|
|             Algiers|    1|
|         Aliso Viejo|    1|
|             Alloway|    2|
|          Almonesson|    1|
|               Alton|  150|
|              Ambler|  211|
|           Andalusia|    3|
|             Antioch|  231|
|        Apollo Beach|  122|
|        Apollo beach|    1|
+--------------------+-----+
only showing top 20 rows



### Finally we take a look at the user dataset which we can join with the other datasets to understand user attributes

In [23]:
user_df.printSchema()
user_df.show(8)
print("Number of yelp users: " + str(user_df.count()))

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)

+-------------+---------------+---------------+----------------+-----

Number of yelp users: 1987897


In [26]:
missingdata_user = user_df.select([count(when(col(c).isNull(), c)).alias(c) for c in user_df.columns]).show()
missingdata_user

+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+----+-----+----+-------+-----+----+------------+------+-------+-------------+
|average_stars|compliment_cool|compliment_cute|compliment_funny|compliment_hot|compliment_list|compliment_more|compliment_note|compliment_photos|compliment_plain|compliment_profile|compliment_writer|cool|elite|fans|friends|funny|name|review_count|useful|user_id|yelping_since|
+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+----+-----+----+-------+-----+----+------------+------+-------+-------------+
|            0|              0|              0|               0|             0|              0|              0|              0|                0|               0|       

In [29]:
user_df.describe('review_count').show()

+-------+------------------+
|summary|      review_count|
+-------+------------------+
|  count|           1987897|
|   mean|23.394409267683386|
| stddev| 82.56699161797889|
|    min|                 0|
|    max|             17473|
+-------+------------------+



#### As we can see, some users have 0 reviews and therefore won't show up in the reviews dataset. It is important to note that users with less reviews may be less credible than users with more reviews. 

#### After assessing the missingness in all three datasets, only the business dataset expressed some nulls. Since this is in the attribute section we can equate a null entry as a business not exhibiting any of the attributes listed above.